In [ ]:
using Pkg, Revise, PyCall
import PyPlot; const plt = PyPlot
Pkg.activate("../..")
using Clapeyron

In [ ]:
x₀ = [0.5,0.5]
T = 298.15
p = 1e5
np = 2
X0 = [0.25,0.7936385866310022,-4.02901049858986,-4.255010578067345,0.5,0.5,1.,1.]
model = PCSAFT(["methanol","cyclohexane"])

F(x) = Obj_HELD_tp_flash(model,p,T,x₀,x,np)
# F(X0)
r = Clapeyron.Solvers.optimize(F,X0)
Clapeyron.Solvers.x_sol(r)

In [ ]:
model = PCSAFT(["methanol","benzene"])
tp_flash(model,1.2e6,433.15,[0.3,0.7],HELDTPFlash(verbose=true,max_steps=15))

In [ ]:
revise(Clapeyron)